In [19]:
import pandas as pd
import re
import glob

# Load CSV and set AccessionID as index for fast lookup
df = pd.read_csv('./med_papers.csv')
df.set_index('AccessionID', inplace=True)

new_list = []

for f in glob.glob("./extracted/*.txt"):
    # Split filename and extract accession ID
    result = re.split(r'[/.]+', f)
    accession_id = result[2]
    row = df.loc[accession_id].to_dict()
    new_row = {'AccessionID':accession_id, 'JournalID':row['JournalID'], 'JIF':row['JIF']}
    #print(new_row)
    new_list.append(new_row)
    

# Convert to DataFrame
new_df = pd.DataFrame(new_list)
print(new_df)


        AccessionID                       JournalID   JIF
0        PMC9439785            0025-7974,1536-5964,   1.4
1        PMC9159206            1368-5031,1742-1241,   2.4
2       PMC10685901  1029-2977,1735-3947,1029-2977,   1.0
3        PMC9718913                      2050-0904,   0.6
4       PMC10759104                      2044-6055,   2.3
...             ...                             ...   ...
118557  PMC10571083            0002-7863,1520-5126,  15.7
118558   PMC9147456                      2077-0383,   2.9
118559   PMC9696955                      2077-0383,   2.9
118560   PMC9701189                      2041-1723,  15.7
118561  PMC10416304            0002-7863,1520-5126,  15.7

[118562 rows x 3 columns]


In [20]:
new_df.to_csv('./extracted_med_papers.csv',index=False)

In [ ]:
import re
import glob
import os

pattern = re.compile(r'[^A-Za-z\s]')

for f in glob.glob("./med_txts/*.txt"):
    with open(f, "r", encoding="utf-8", errors="ignore") as infile:
        text = infile.read()

    cleaned = pattern.sub("", text)

    new_name = os.path.join("cleaned", os.path.basename(f))
    os.makedirs("cleaned", exist_ok=True)

    with open(new_name, "w", encoding="utf-8") as outfile:
        outfile.write(cleaned)

In [ ]:
import os
import glob

def extract_section(lines):
    in_body = False
    output = []

    for line in lines:
        stripped = line.strip()

        # Find the Body line (exact match)
        if not in_body:
            if stripped == "Body":
                in_body = True
            continue  # skip everything before AND the Body line itself

        # Once inside the body, stop at References line (exact match)
        if stripped == "References":
            break

        output.append(line)

    return "".join(output)


os.makedirs("extracted", exist_ok=True)

for f in glob.glob("./cleaned/*.txt"):
    with open(f, "r", encoding="utf-8", errors="ignore") as infile:
        lines = infile.readlines()

    extracted = extract_section(lines)

    out_path = os.path.join("extracted", os.path.basename(f))
    with open(out_path, "w", encoding="utf-8") as outfile:
        outfile.write(extracted)
